In [1]:
import cv2
import numpy as np
import tensorflow as tf
import os
from pandas import DataFrame

from sklearn.cluster import KMeans
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import cv2
import argparse
import webcolors
import numpy as np
import cv2
import shutil

import PIL
from PIL import Image 
from PIL import ImageEnhance 

from pandas import ExcelWriter
import pandas as pf
import pyforms
from   pyforms          import BaseWidget
from   pyforms.controls import ControlText
from   pyforms.controls import ControlButton
from tkinter import *
from tkinter import filedialog, messagebox, PhotoImage
from tkinter.filedialog import askopenfilename,askdirectory

In [ ]:
class window(): 
    def __init__(self):
        self.root = Tk()
        self.folder_path_fotos = str()
        self.folder_path = str()
        self.excel_path = str()
        self.imagem_ind=str()
        self.matricula = []
        self.matricula_n_unique = []
        self.modelo_n_unique = []
        self.l1 = []
        self.path_ = str()
        #existe = []
        self.modelo = []
        self.cor = []
        self.cor_n_unique = []
        self.form()
        #print(self.l1)
        self.root.mainloop()
        
    def closest_colour(self,requested_colour):
        min_colours = {}
        for key, name in webcolors.HTML4_HEX_TO_NAMES.items():
            r_c, g_c, b_c = webcolors.hex_to_rgb(key)
            rd = (r_c - requested_colour[0]) ** 2
            gd = (g_c - requested_colour[1]) ** 2
            bd = (b_c - requested_colour[2]) ** 2
            min_colours[(rd + gd + bd)] = name
        return min_colours[min(min_colours.keys())]

    def get_colour_name(self,requested_colour):
        try:
            closest_name = actual_name = webcolors.rgb_to_name(requested_colour,spec = 'html4')
        except ValueError:
            closest_name = self.closest_colour(requested_colour)
            actual_name = None
        return actual_name, closest_name

    # load the image and convert it from BGR to RGB so that
    # we can dispaly it with matplotlib
    #def crop_image(self):
     #   image = cv2.imread(self.path_)
      #  center = image.shape
       # y=500
        #x=300
        #h=500
        #w=1500
        #print(h,w)
        #crop_img=image[y:y+h,x:x+w]
        #self.crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
        # show our image
        #plt.figure()
        #plt.axis("off")
        #plt.imshow(self.crop_img)
        #return self.image

    # import the necessary packages
    def centroid_histogram(self,clt):
        # grab the number of different clusters and create a histogram
        # based on the number of pixels assigned to each cluster
        numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
        (hist, _) = np.histogram(clt.labels_, bins = numLabels)
        # normalize the histogram, such that it sums to one
        hist = hist.astype("float")
        hist /= hist.sum()
        # return the histogram
        return hist

    def plot_colors(self,hist, centroids):
        # initialize the bar chart representing the relative frequency
        # of each of the colors
        bar = np.zeros((50, 300, 3), dtype = "uint8")
        startX = 0
        # loop over the percentage of each cluster and the color of
        # each cluster
        max = 0
        max_colour = 0.000
        for (percent, color) in zip(hist, centroids):
            # plot the relative percentage of each cluster
            if percent > max:
                max = percent
                max_colour = self.get_colour_name(color.astype("uint8").tolist())
            #print(get_colour_name(color.astype("uint8").tolist()),percent)
            endX = startX + (percent * 300)
            cv2.rectangle(bar, (int(startX), 0), (int(endX), 50),
                color.astype("uint8").tolist(), -1)
            startX = endX
        # return the bar chart
        # show our color bart
        plt.figure()
        plt.axis("off")
        plt.imshow(bar)
        plt.show()
        print(max_colour[1])
        return bar, max_colour

    
    def CNN(self,path):
        # Previsão através de uma imagem
        #definir tamanho das imagens
        nlinhas = 256
        ncolunas = 256
        canais = 3  #dimensões das cores (1 para preto e branco e 3 para colorido)
        print(path)
        # Recreate the exact same model, including its weights and the optimizer
        model = tf.keras.models.load_model("C:/Users/joaog/OneDrive/Ambiente de Trabalho/outros/mercedes/Dior/new CNN/marca_modelo_sgd.h5")
        imagem_cor = cv2.resize(cv2.imread(path, cv2.IMREAD_COLOR), (nlinhas,ncolunas), interpolation=cv2.INTER_CUBIC)
        #imagem_peb = cv2.cvtColor(imagem_cor, cv2.COLOR_BGR2GRAY)
        #tf.expand_dims(
        imagem = tf.expand_dims(np.array(imagem_cor).astype("float32") / 255, 0)   
            #,3)
        pred = model.predict(imagem)
        pred = np.argmax(pred)
        marcas = ['ClasseA','ClasseE','ForFour']
        #plt.figure(figsize=(5,2.5))
        #plt.imshow(imagem_cor)
        
        #image = PIL.Image.open(path)
        #image = ImageEnhance.Color(image) 
        #image = image.enhance(2) 
        #image = ImageEnhance.Brightness(image) 
        #image = image.enhance(0.5) 
        #image.save(path)
    
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        y=600
        x=600
        h=200
        w=1000
        #print(h,w)
        crop_img=image[y:y+h,x:x+w]
        #crop_img = self.increase_brightness(crop_img)
        
        
        # show our image
        plt.figure()
        plt.axis("off")
        plt.imshow(image)
        #return self.image
        
        
        # reshape the image to be a list of pixels
        crop_img = crop_img.reshape((crop_img.shape[0] * crop_img.shape[1], 3))
        # cluster the pixel intensities
        clt = KMeans(n_clusters =5)
        clt.fit(crop_img)
    
        # build a histogram of clusters and then create a figure
        # representing the number of pixels labeled to each color
        hist = self.centroid_histogram(clt)
        bar, max_colour = self.plot_colors(hist, clt.cluster_centers_)
    
        # return the bar chart
        # show our color bart
        plt.figure()
        plt.axis("off")
        plt.imshow(bar)
        plt.show()
    
        return marcas[pred],max_colour[1]

    def imagem_in_dir(self):
        
        dir_=os.listdir(self.folder_path_fotos)
        for imagem in dir_:
            modelo_1, cor_1 = self.CNN(self.folder_path_fotos + "/" + imagem)
            print(modelo_1)
            imagem = imagem.split('.')[0]
            self.matricula_n_unique.append(imagem)
            self.cor_n_unique.append(cor_1)
            self.modelo_n_unique.append(modelo_1)
            if [modelo_1,cor_1] not in self.l1:
                self.l1.append([modelo_1,cor_1])
                self.modelo.append(modelo_1)
                self.cor.append(cor_1)
                self.matricula.append(imagem)
                shutil.copy(self.folder_path_fotos + "/" + imagem + '.JPG',self.file_path_jafeito)
                print(self.folder_path_fotos + "/" + imagem + '.JPG',self.file_path_jafeito)

    def write_excel(self):
        df = DataFrame({'matrículas':self.matricula, 'modelo':self.modelo, 'cor':self.cor})
        with ExcelWriter(self.excel_path) as writer:  
            df.to_excel(writer, sheet_name='Sheet1')
 

    def focus1(self,event):
        # set focus on the course_field box
        self.folder_path_fotos = filedialog.askdirectory(title = 'open')
        self.folder_path_fotos = str(self.folder_path_fotos)
        self.path_field_fotos.insert(0,self.folder_path_fotos)
        print(self.folder_path_fotos)

    def focus5(self,event):
        # set focus on the course_field box
        self.file_path_jafeito = filedialog.askdirectory(title = 'open')
        self.file_path_jafeito = str(self.file_path_jafeito)
        self.path_field_folder.insert(0,self.file_path_jafeito)
        print(self.file_path_jafeito)

    def focus8(self,event):
        # set focus on the course_field box
        self.excel_path = filedialog.askopenfilename(title = 'open')
        self.excel_path = str(self.excel_path)
        self.path_field_excel.insert(0,self.excel_path)
        print(self.excel_path)

    def focus3(self,event):
        self.path_field_fotos.delete(0,END)
        self.path_field_folder.delete(0,END)
        self.path_field_excel.delete(0,END)
        self.folder_path_fotos = ""
        self.folder_path = ""
        self.excel_path = ""

    def focus2(self,event):
        modelo,cor = self.CNN(self.path_)
        if [modelo,cor] in self.l1:
            return messagebox.Message('A viatura {} de cor {} já está na coleção'.format(modelo,cor))
        else:
            self.l1.append([modelo,cor])
            return messagebox.Message('A viatura {} de cor {} foi adicionada à coleção'.format(modelo,cor))
        #como ir buscar imagem


    def focus4(self,event):
        self.l1 = []
        self.read_excel()
        self.imagem_in_dir()
        #print(self.l1,self.modelo,self.matricula,self.cor)
        #print(len(self.l1),len(self.modelo),len(self.matricula),len(self.cor))
        self.write_excel()
        
    
    def read_excel(self):
        import pandas as pd 
        df = pd.read_excel(self.excel_path)
        self.matricula = df["matrículas"].tolist()
        self.modelo = df["modelo"].tolist()
        self.cor = df["cor"].tolist()
        for i in range(len(self.modelo)):
            self.l1.append([self.modelo[i],self.cor[i]])
        #print(self.l1,self.modelo,self.matricula,self.cor)

    def focus6(self,event):
        self.imagem_in_dir()
        self.write_excel()
    
    def form(self):
        self.root.configure(background='dark blue')
        self.root.title("Maia Tarot")
        self.root.geometry("800x400")
        
        
        self.path = Label(self.root,text='Fotos',bg='white')
        self.path.grid(row = 1, column = 0,sticky = N,padx = 20,pady = 20)
        self.path_field_fotos = Entry(self.root, textvariable = self.folder_path_fotos) 
        self.path_field_fotos.bind("<Button>",self.focus1)
        self.path_field_fotos.grid(row=1,column=1,ipadx='100',sticky = N,padx = 20,pady = 20)
        
        self.path_field = Label(self.root,text='Folder',bg='white')
        self.path_field.grid(row = 2, column = 0,sticky = N,padx = 20,pady = 20)
        self.path_field_folder = Entry(self.root, textvariable = self.folder_path) 
        self.path_field_folder.bind("<Button>",self.focus5)
        self.path_field_folder.grid(row=2,column=1,ipadx='100',sticky = N,padx = 20,pady = 20)

        self.path_excel = Label(self.root,text='Excel',bg='white')
        self.path_excel.grid(row = 3, column = 0,sticky = N,padx = 20,pady = 20)
        self.path_field_excel= Entry(self.root, textvariable = self.excel_path) 
        self.path_field_excel.bind("<Button>",self.focus8)
        self.path_field_excel.grid(row=3,column=1,ipadx='100',sticky = N,padx = 20,pady = 20)

        
        self.button3 = Button(self.root,text = 'Atualizar catálogo',fg = 'Black',bg='Grey')
        self.button3.bind('<Button>',self.focus4)
        self.button3.grid(row=4, column=0,sticky = W)
        
        self.button6 = Button(self.root,text = 'Novo Catálogo',fg = 'Black',bg='Grey')
        self.button6.bind('<Button>',self.focus6)
        self.button6.grid(row=4, column=1,sticky = W)

        self.button7 = Button(self.root,text = 'Reset',fg = 'Black',bg='Grey')
        self.button7.bind('<Button>',self.focus3)
        self.button7.grid(row=3, column=2,sticky = W)

        self.text = Text(self.root,height = 8)
        self.text.grid(row = 5,column = 1)
        self.text.insert('1.0','Novo Catálogo: No campo \Fotos\, selecionar a pasta das imagens de estúdio que queremos adicionar ao catálogo\n')
        self.text.insert('2.0','No campo \Pasta\, escolher a pasta em que queremos guardar o novo catálogo\n')
        self.text.insert('3.0','No campo \Excel\, escolher o ficheiro Excel(xlsx) em que queremos registar o novo catálogo\n')
        self.text.insert('4.0','Atualizar Catálogo: Manter a escolha do campo de \Fotos\ \n')
        self.text.insert('6.0','No campo \Excel\, escolher o ficheiro xlsx em que o antigo catálogo está registado\n')
        self.text.insert('5.0','No campo \Pasta\, escolher a pasta do antigo Catálogo\n')
        
    
            

    
if __name__ == '__main__':
    win = window()


    


In [ ]:
df = DataFrame({'modelo':modelo, 'cor':cor})
df.to_excel('C:/Users/joaog/OneDrive/Ambiente de Trabalho/outros/mercedes/Fotos.xlsx', sheet_name='Sheet1', index=False)